# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


#headers for connecting to amazon
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

#url for apple charger to fetch reviews
amazon_reviews_url = 'https://www.amazon.com/Apple-20W-USB-C-Power-Adapter/product-reviews/B08L5M9BTJ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

#took 2000 pages so that we get min of 1000 reviews for sure
pages_of_reviews = 2000


#this function just the url and takes html of all the pages
def reviewsHtml(url, len_page):

    bs = []

    for page_no in range(1, pages_of_reviews + 1):

        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }

        response = requests.get(url, headers=headers)
        s = BeautifulSoup(response.text, 'lxml')
        bs.append(s)

    return bs


#we get reviews from this function, we use css classes to identify the review title, desc date etc.,
def getReviews(data):

    review_dict = []
    hooks = data.select('div[data-hook="review"]')

    for hook in hooks:

        try:
            name = hook.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            rating = hook.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            rating = 'N/A'

        try:
            title = hook.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            datetime_str = hook.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = hook.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        data_dict = {
            'Name' : name,
            'Rating' : rating,
            'Title' : title,
            'Date' : date,
            'Desc' : description
        }

        review_dict.append(data_dict)

    return review_dict


block_data = reviewsHtml(amazon_reviews_url, pages_of_reviews)

reviews = []

#we append the data to reviews list
for data in block_data:
    review = getReviews(data)
    reviews += review

df_reviews = pd.DataFrame(reviews)

print(df_reviews)

#storing the reviews in a csv file
df_reviews.to_csv('amazonreviews.csv', index=False)

                                                   Name Rating  \
0                                         SwamyLakamana    5.0   
1                                       Reviews by Andy    5.0   
2                                               Shirley    5.0   
3                                          Justin Huber    5.0   
4                                       Amazon Customer    5.0   
...                                                 ...    ...   
2195                                        M. Jennings    5.0   
2196                                  Christopher Pitts    4.0   
2197                                              Diana    5.0   
2198                                              Marty    1.0   
2199  NLine….. Not at all what expected failed to wo...    5.0   

                                                  Title        Date  \
0           5.0 out of 5 stars\nAuthentic Apple Adapter  17/11/2023   
1     5.0 out of 5 stars\nPower in a Compact Package...  25/01/20

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

df_reviews = pd.read_csv('amazonreviews.csv')

#we use regular expresiions for identifying the words and then find stop words using stopwords library
# porterstemmer for stemming and wordnetlemmatizer for lemmetizing the sentence
def clean_text(txt):
    txt = re.sub(r'[^a-zA-Z\s]', '', txt)
    txt = re.sub(r'\d+', '', txt)
    txt = txt.lower()
    stwords = set(stopwords.words('english'))
    wds = txt.split()
    fwds = [word for word in wds if word not in stwords]
    txt = ' '.join(fwds)
    ps = PorterStemmer()
    stmwds = [ps.stem(word) for word in fwds]
    txt = ' '.join(stmwds)
    lmtze = WordNetLemmatizer()
    lmt_wrds = [lmtze.lemmatize(word) for word in fwds]
    txt = ' '.join(lmt_wrds)
    return txt

df_reviews[''] = df_reviews['Desc'].apply(clean_text)
df_reviews.to_csv('cleaned_amazon_reviews.csv', index=False)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [7]:
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

df_cleaned_reviews = pd.read_csv('cleaned_amazon_reviews.csv')

def pos_tagging(text):
    tkns = word_tokenize(text)
    pos_tags = pos_tag(tkns)
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

    #this for loop identifies the pos of a word using pos)tag function
    for word, pos in pos_tags:
        if pos.startswith('N'):
            pos_counts['Noun'] += 1
        elif pos.startswith('V'):
            pos_counts['Verb'] += 1
        elif pos.startswith('J'):
            pos_counts['Adjective'] += 1
        elif pos.startswith('R'):
            pos_counts['Adverb'] += 1
    return pos_counts
#initially
total_pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

#go through all the words in the reviews and identify its pos
for index, row in df_cleaned_reviews.iterrows():
    review_text = row['Desc']
    # Calculate POS counts for the current review
    pos_counts = pos_tagging(review_text)
    # Accumulate counts for each POS category
    for pos_category, count in pos_counts.items():
        total_pos_counts[pos_category] += count

# Print total counts of POS categories
print("Total Counts of POS Categories:")
print(total_pos_counts)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Total Counts of POS Categories:
{'Noun': 147620, 'Verb': 91960, 'Adjective': 47080, 'Adverb': 29040}


In [8]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tree import Tree

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

df_cleaned_reviews = pd.read_csv('cleaned_amazon_reviews.csv')

#Constituency parsing aims to identify the syntactic structure of sentences
#based on the rules of a formal grammar.
def cptree(text):
    tkns = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tkns)
    grammar = "NP: {<DT>?<JJ>*<NN>}"
    cp = nltk.RegexpParser(grammar)
    parsed_sentence = cp.parse(tagged_tokens)
    print("CPT:")
    parsed_sentence.pretty_print()

#shows dependencies between the words
def dptree(text):
    tkns = word_tokenize(text)
    pos_tags = nltk.pos_tag(tkns)
    grammar = "NP: {<DT>?<JJ>*<NN>}"
    cp = nltk.RegexpParser(grammar)
    parsed_sentence = cp.parse(pos_tags)
    print("DPT:")
    print(parsed_sentence)

#taking first review as an example to show
example_review = df_cleaned_reviews.iloc[0]['Desc']

cptree(example_review)
dptree(example_review)


Constituency Parsing Tree:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

df_cleaned_reviews = pd.read_csv('cleaned_amazon_reviews.csv')

# Named Entity Recognition by splitting words and identifying pos
def named_entity_recognition(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens) #parts of speech identification
    ne_tags = nltk.ne_chunk(pos_tags) #entity chuncking
    # Initialize empty list to store entities
    entities = []
    # Iterate over the named entity chunks and finds entity name and labels
    for chunk in ne_tags:
        if isinstance(chunk, nltk.Tree):
            entity = " ".join([token for token, pos in chunk.leaves()])
            label = chunk.label()
            if label == 'PERSON' or label == 'ORGANIZATION' or label == 'LOCATION' or label == 'DATE':
                entities.append((entity, label))
    return entities

all_entities = []
for index, row in df_cleaned_reviews.iterrows():
    review_text = row['Desc']
    entities = named_entity_recognition(review_text)
    all_entities.extend(entities)

# Create DataFrame to store entities and their counts
df_entities = pd.DataFrame(all_entities, columns=['Entity', 'Label'])
entity_counts = df_entities['Label'].value_counts()

# Print counts of each entity type
print("Entity Counts:")
print(entity_counts)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Entity Counts:
ORGANIZATION    9020
PERSON          8580
Name: Label, dtype: int64


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# I have learnt how to scrape rewuied data from a website using beautifulsoup
# I have learnt how to use the techniques like lemmatization, stemming and stop
#words recognition which help me understand how information retireval works
# I got to improve my knowlegede on few more information retrival and NLP methoods like POS reconition, named entity